In [ ]:
# Model without feature selection

In [24]:
import numpy as np
import pandas as pd
from google.colab import drive 
from sklearn.preprocessing import LabelEncoder
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import Lasso
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn import metrics
import pickle

In [6]:
# import full dataset
drive.mount('/content/drive')
dataset = pd.read_csv(("/content/drive/MyDrive/MLOPS/dementia prediction/trainData.csv"))
print("input data shape:",dataset.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
input data shape: (29876, 103)


In [10]:
# extract features and target set from dataset

x = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

In [11]:
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(len(y),1) 

In [12]:
# standardize feature and target set

sslr= StandardScaler() 
x = sslr.fit_transform(x) 
y = sslr.fit_transform(y)

In [19]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state=0) 
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=0)

In [ ]:
# function with the constructed artifical neural network

def ANN(optimizer = 'adam',neurons_1=32,neurons_2=32,batch_size=32,epochs=50,learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(neurons_1, input_shape=(x_train.shape[1],), activation='relu'))
    model.add(Dense(neurons_2, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse',metrics=['mse'])
    history = model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_val, y_val)) 
    def plot_loss(history):
      plt.plot(history.history['loss'], label='loss')
      plt.plot(history.history['val_loss'], label='val_loss')
      plt.ylim([0,10])
      plt.xlabel('Epoch')
      plt.ylabel('Error (Loss)')
      plt.legend()
      plt.grid(True)
    ########################################################
    plot_loss(history)
    return model

clf =ANN()

In [ ]:
rf_params = {
    'optimizer': ['adam','rmsprop'], 
    'neurons_1': list(np.arange(16,96,16)),
    'neurons_2': list(np.arange(16,96,16)),
    'batch_size': [16,32,64],
    'epochs':[10,20,30,40,50],
    'learning_rate': [1e-1,1e-2,1e-3,1e-4]
}
n_iter_search=10
clf = KerasRegressor(build_fn=ANN, verbose=0)
random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
random.fit(x_train, y_train)
print(random.best_params_)
randomParams = random.best_params_
print("MSE:"+ str(-random.best_score_))

In [34]:
# save the model with the best parameters
Pkl_Filename = "Pickle_RL_Model.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(randomParams, file)

# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)
Pickled_LR_Model

{'batch_size': 16,
 'epochs': 50,
 'learning_rate': 0.001,
 'neurons_1': 32,
 'neurons_2': 80,
 'optimizer': 'rmsprop'}